In [1]:
import transformers
import textattack
import datasets
from datasets import load_dataset, DatasetDict, load_from_disk



/Users/singh.akash/opt/anaconda3/envs/adversarial/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
datasets.__version__

'1.18.4'

In [4]:
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-uncased",)
model = transformers.AutoModelForSequenceClassification.from_pretrained("artemis13fowl/bert-base-uncased-imdb")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)


In [6]:
data = load_dataset("artemis13fowl/imdb",split= "attack_eval_truncated")
data

Using custom data configuration artemis13fowl--imdb-f77fd77a6b2e946b
Reusing dataset parquet (/Users/singh.akash/.cache/huggingface/datasets/parquet/artemis13fowl--imdb-f77fd77a6b2e946b/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


Dataset({
    features: ['text', 'label'],
    num_rows: 100
})

In [7]:
dataset = textattack.datasets.HuggingFaceDataset(data)

In [8]:
attack = textattack.attack_recipes.TextFoolerJin2019.build(model_wrapper)

# Attack 20 samples with CSV logging and checkpoint saved every 5 interval

attack_args = textattack.AttackArgs(num_examples=1, log_to_csv="../output/textattack_pruthi.csv", checkpoint_interval=5, checkpoint_dir="checkpoints", disable_stdout=True)

attacker = textattack.Attacker(attack, dataset, attack_args)

attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path log.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

  0%|                                                                                                                                                                                                                                                   | 0/1 [00:00<?, ?it/s]2022-03-24 14:45:28.506275: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [02:14<00:00, 134.16s/it]/Users/singh.akash/opt/anaconda3/envs/adversarial/lib/python3.8/site-packages/textattack/loggers/csv_logger.py:41: Fut


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 0.88%  |
| Average num. words per input: | 113.0  |
| Avg num queries:              | 130.0  |
+-------------------------------+--------+
